In [1]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint
import keras.metrics
import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
sequence_length = 15 # timestep

In [3]:
path_save = "Extract_data"
f_x = open(path_save+"/train_x.pickle",'rb')
f_y = open(path_save+"/train_y.pickle",'rb')
origin_train_x = pickle.load(f_x)
train_y = pickle.load(f_y)

f_x = open(path_save+"/test_x.pickle",'rb')
f_y = open(path_save+"/test_y.pickle",'rb')
origin_test_x = pickle.load(f_x)
test_y = pickle.load(f_y)

### Sampling Function

In [4]:
def sampling_x(x, is_training):
    
    frames = x
    
    # Start index and sample interval for the test set
    start_i, sample_interval = 0, len(frames) // sequence_length
    if is_training:
        # Randomly choose sample interval and start frame
        sample_interval = np.random.randint(1, len(frames) // sequence_length + 1)
        start_i = np.random.randint(0, len(frames) - sample_interval * sequence_length + 1)

    # Extract frames as tensors
    image_sequence = []
    for i in range(start_i, len(frames), sample_interval):
        # image_path = frames[i]
        if len(image_sequence) < sequence_length:
            image_sequence.append(frames[i])
        else:
            break
    image_sequence = np.array(image_sequence)   
    return image_sequence


def reform_to_sequence(data_x, is_training):
    
    train_x = np.zeros((len(data_x), sequence_length, data_x[0].shape[-1]) ) #(len,timestep, 28)
    for i,x in enumerate(data_x):
        sequence = sampling_x(x, is_training)
        train_x[i] = sequence       
        
    return train_x     


### Model Define

In [5]:
number_feature = origin_train_x[0].shape[-1] # last index
model = Sequential()
model.add(CuDNNLSTM(50, input_shape=(sequence_length, number_feature),return_sequences=True))
model.add(Dropout(0.6))#使用Dropout函数可以使模型有更多的机会学习到多种独立的表征
model.add(CuDNNLSTM(50, input_shape=(sequence_length, number_feature)) )
model.add(Dropout(0.6))
model.add(Dense(4, activation='softmax'))
sgd = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,)
callbacks_list = [checkpoint]

W0920 16:36:00.315918 140433912616768 deprecation.py:506] From /home/peachman/Documents/environments/action_reg/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0920 16:36:00.562067 140433912616768 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0920 16:36:00.765580 140433912616768 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


### Prepare Test Set

In [6]:
test_x = reform_to_sequence(origin_test_x, is_training=False)

### Train

In [7]:
num_epoch = 100

for i_ep in range(num_epoch):
    
    print('epoch: ', i_ep)
    train_x = reform_to_sequence(origin_train_x, is_training=True)
    model.fit(train_x, train_y, epochs=1,
             validation_data=(test_x,test_y), callbacks=callbacks_list)
    print("-----------------------")



epoch:  0
Train on 5632 samples, validate on 2413 samples
5568/5632 [============================>.] - ETA: 0s - loss: 1.1037 - acc: 0.5135
Epoch 00001: val_acc improved from -inf to 0.79362, saving model to weights-improvement-01-0.79.hdf5
5632/5632 [==============================] - 14s 3ms/sample - loss: 1.1004 - acc: 0.5158 - val_loss: 0.8055 - val_acc: 0.7936
-----------------------
epoch:  1
Train on 5632 samples, validate on 2413 samples
5600/5632 [============================>.] - ETA: 0s - loss: 0.8181 - acc: 0.6607
Epoch 00001: val_acc improved from 0.79362 to 0.92872, saving model to weights-improvement-01-0.93.hdf5
5632/5632 [==============================] - 2s 339us/sample - loss: 0.8164 - acc: 0.6619 - val_loss: 0.4561 - val_acc: 0.9287
-----------------------
epoch:  2
Train on 5632 samples, validate on 2413 samples
5568/5632 [============================>.] - ETA: 0s - loss: 0.6660 - acc: 0.7493
Epoch 00001: val_acc improved from 0.92872 to 0.96395, saving model to wei

-----------------------
epoch:  23
Train on 5632 samples, validate on 2413 samples
5600/5632 [============================>.] - ETA: 0s - loss: 0.1877 - acc: 0.9368
Epoch 00001: val_acc did not improve from 0.99917
5632/5632 [==============================] - 1s 253us/sample - loss: 0.1877 - acc: 0.9368 - val_loss: 0.0604 - val_acc: 0.9793
-----------------------
epoch:  24
Train on 5632 samples, validate on 2413 samples
5600/5632 [============================>.] - ETA: 0s - loss: 0.1990 - acc: 0.9379
Epoch 00001: val_acc did not improve from 0.99917
5632/5632 [==============================] - 1s 251us/sample - loss: 0.2008 - acc: 0.9366 - val_loss: 0.1451 - val_acc: 0.9834
-----------------------
epoch:  25
Train on 5632 samples, validate on 2413 samples
5408/5632 [===========================>..] - ETA: 0s - loss: 0.2372 - acc: 0.9186
Epoch 00001: val_acc did not improve from 0.99917
5632/5632 [==============================] - 1s 251us/sample - loss: 0.2374 - acc: 0.9190 - val_loss:

KeyboardInterrupt: 